In order to run this notebook, you need to install the following packages:

- `requests`
- `tqdm`
- `matplotlib`
- `scipy`
- `xarray-leaflet`

The recommended way is: `conda install -c conda-forge requests tqdm matplotlib scipy xarray-leaflet`

Note: you can replace `conda` with [`mamba`](https://github.com/QuantStack/mamba) if you don't like to wait ;-)

In [ ]:
import requests
import os
from tqdm import tqdm
import zipfile
import xarray as xr
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
import xarray_leaflet
from ipyleaflet import Map, basemaps

We first download the awesome [HydroSHEDS](https://hydrosheds.org) dataset, and in particular the flow accumulation for South America. You can think of flow accumulation as a potential river flow, so we will have a visual representation of rivers, including the great Amazon river.

In [ ]:
url = 'https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/hydrosheds/sa_30s_zip_grid/sa_acc_30s_grid.zip'
filename = os.path.basename(url)
name = filename[:filename.find('_grid')]
adffile = name + '/' + name + '/w001001.adf'

if not os.path.exists(adffile):
    r = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        total_length = int(r.headers.get('content-length'))
        for chunk in tqdm(r.iter_content(chunk_size=1024), total=(total_length/1024) + 1):
            if chunk:
                f.write(chunk)
                f.flush()
    zip = zipfile.ZipFile(filename)
    zip.extractall('.')

Let's open the box and see what's in there.

In [ ]:
da = xr.open_rasterio(adffile)
da

We can see that the projection is `EPSG:4326` (aka `WGS84`), which is the only projection currently supported by `xarray-leaflet`. Here the coordinate `x` corresponds to longitudes, and `y` to latitudes. There `band` coordinate is pretty useless. The only preprocessing we will do is replacing `NaN` values with zeros.

In [ ]:
nan = da.attrs['nodatavals'][0]
da = da.sel(band=1)
da = xr.where(da==nan, 0, da)

`xarray-leaflet` has two modes, one is for static maps and the other for dynamic maps. By dynamic we mean that you won't see the same thing depending on the current map view. For instance, if you zoom in you might want to show details that you would otherwise be hidden. So the map really adapts to where you look on the Earth. The downside of dynamic maps is that you will see more flickering as you interact with the map, because tiles have to be refreshed as soon as you drag or zoom.

On the contrary, static maps won't be as reactive to the location. Of course they will refine as you zoom in, but they won't change dramatically. On the other hand, they are rendered really smoothly.

This example can be configured to work both in static or dynamic mode. You can change the following `dynamic` variale to `True` or `False` to switch between the two modes (you will have to re-execute the cells to take it into account).

In [ ]:
# you can change the value of `dynamic` to True or False and see the difference in the interaction with the map

dynamic = True
if not dynamic:
    global_vmin = np.min(da).values
    global_vmax = np.max(da).values

There are 3 stages in `xarray-leaflet` where you can apply transformations to your data. The first one operates on the visible part. Often, this is where you want to apply dynamic transformations. Transformations are Python functions accepting the data as first argument and optionally parameters from the previous transformations. Because `transform0` is the first transformation, there is no other input parameters appart from the `DataArray`. Here we return the minimum and maximum values present in the data.

In [ ]:
def transform0(da):
    if dynamic:
        vmin = np.min(da).values
        vmax = np.max(da).values
    else:
        vmin = global_vmin
        vmax = global_vmax
    return da, vmin, vmax

The second transformation applies to the data contained in each Leaflet tile before reprojection. Reprojection needs your data to fit into memory, so you may want to downsample your data and keep approximately the same number of points as there are in a tile (256 x 256). In this transformation you also want to have you data normalized between 0 and 1, which we can do thanks to the `vmin` and `vmax` values that we got from the previous transformation.

In [ ]:
def transform1(xr_tile, vmin, vmax):
    ny, nx = xr_tile.shape
    wx = nx // (256 // 2)
    wy = ny // (256 // 2)
    if wx > 0 and wy > 0:
        xr_tile = xr_tile.coarsen(x=wx, y=wy, boundary="trim").max()
    xr_tile = (xr_tile - vmin) / (vmax - vmin)
    return xr_tile

In the third and last transformation, the data is reprojected to Web Mercator (the projection used in Leaflet by default). This is the last opportunity to transform your data before saving it to a `png` file. That's often when you want to apply styling. Here we make the rivers appear thicker and we reduce the amplitude with a square root. You also need to choose a colormap.

In [ ]:
def transform2(np_tile):
    radius = 2
    circle = np.zeros((2*radius+1, 2*radius+1)).astype('uint8')
    y, x = np.ogrid[-radius:radius+1,-radius:radius+1]
    index = x**2 + y**2 <= radius**2
    circle[index] = 1
    np_tile = np.sqrt(np_tile)
    np_tile = scipy.ndimage.maximum_filter(np_tile, footprint=circle)
    np_tile = plt.cm.inferno(np_tile)
    return np_tile

We are almost done, we just need to create a map before passing it to our `DataArray`.

In [ ]:
m = Map(center=[-20, -60], zoom=3, basemap=basemaps.CartoDB.DarkMatter)
m

To show our data on the map, we call `.leaflet.plot()` on our `DataArray`, and pass as parameters the map, the name on the lat/lon dimensions, the transformation functions, and the `dynamic` value. We get back a layer, that we can further control with e.g. a slider to set the opacity.

In [ ]:
l = da.leaflet.plot(m, lat_dim='y', lon_dim='x',
                    transform0=transform0,
                    transform1=transform1,
                    transform2=transform2,
                    dynamic = dynamic)
l.interact(opacity=(0.0,1.0,0.1))

You can now drag and zoom, explore different locations, and if you chose `dynamic=True` you should see details being revealed.